In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [2]:
# Define target url. Search for houses in Copenhagen and surroundings, boligtype = Rent
url = "https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/"
response = requests.get(url)

In [3]:
response.status_code

200

In [4]:
soup = BeautifulSoup(response.text,'html.parser')

In [5]:
soup

<!DOCTYPE html>

<html lang="da">
<head>
<title>Lejelejlighed på DBA - Find billige Lejligheder til leje i hele landet!</title>
<meta charset="utf-8"/>
<link href="https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/" rel="canonical"/>
<link href="https://dbastatic.dk/Content/dba.ico?2.0.0.0-fe77319bf20deb0775cfa5c13a2a76d83b2d4304" rel="shortcut icon" type="image/x-icon"/>
<link href="https://dbastatic.dk/Content/dba.ico?2.0.0.0-fe77319bf20deb0775cfa5c13a2a76d83b2d4304" rel="icon" type="image/x-icon"/>
<meta content="width=960" name="viewport"/>
<meta content="Find den perfekte Lejelejlighed på DBA. Kæmpe udvalg af ledige lejligheder til udleje over hele landet. Find nemt en billig lejelejlighed på DBA i dag!" name="description"/>
<link href="https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-2/" rel="next">
<link as="font" crossorigin="" href="//dbastatic.dk/Content_uex/less/fonts/ProximaNova/DBA_regular.woff" rel="preload" type="fo

In [6]:
# Find all the ads that appear in the first page (so 23 ads in the first search/result page)
ads=soup.find_all("tr",{"class":["dbaListing listing topListing","dbaListing listing hasInsertionFee topListing","dbaListing listing hasInsertionFee","dbaListing listing lastListing","dbaListing listing"]})
len(ads)

22

In [7]:
# Get the link of the first ad, in the first page
ads[0].find("a",{"class":"listingLink"})["href"]

'https://www.dba.dk/3500-3-vaer-lejlighed-69-m2/id-1110918131/'

In [8]:
# Lets get all the links in the first page
links_ads_first_page = []
for i in range(0,len(ads)):
    links_ads_first_page.append(ads[i].find("a",{"class":"listingLink"})["href"])

In [9]:
links_ads_first_page

['https://www.dba.dk/3500-3-vaer-lejlighed-69-m2/id-1110918131/',
 'https://www.dba.dk/1658-4-vaer-lejlighed-81-m2/id-1111184498/',
 'https://www.dba.dk/2300-1-vaer-lejlighed-18-m2/id-1111183325/',
 'https://www.dba.dk/3-vaerelses-lejlighed-i-koebe/id-509876410/',
 'https://www.dba.dk/4-vaer-eller-stoerre-i-koeben/id-509876407/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-koebe/id-510023270/',
 'https://www.dba.dk/4-vaer-eller-stoerre-i-konge/id-508806058/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-roedo/id-510023210/',
 'https://www.dba.dk/3-vaerelses-lejlighed-i-koebe/id-510023153/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-koebe/id-510023154/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-kong/id-510023141/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-kong/id-510023139/',
 'https://www.dba.dk/2-vaerelses-lejlighed-i-broen/id-510023136/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-koebe/id-510023119/',
 'https://www.dba.dk/3-vaerelses-lejlighed-i-hvid/id-510023112/',
 '

In [10]:
# Get the total number of result pages (pagination). So, no we have 100 pages
total_nr_pages=int(soup.find("div",{"class":"pagination pagination-right pagination-modern"}).find_all("a",{"class":"trackClicks a-page-link"})[-1].get_text().replace('\r\n','').strip())
total_nr_pages

100

# Get all the links of pages 

In [11]:
links_of_pages = []
for i in range(1,total_nr_pages+1):
    if i == 1:
        links_of_pages.append("https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/")
    else:
        links_of_pages.append("https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-{}/".format(i))

In [12]:
links_of_pages

['https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-2/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-3/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-4/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-5/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-6/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-7/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-8/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-9/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-10/',
 'https://www.dba.dk/boliger/lejebolig/lejelejlighed/reg-koebenhavn-og-omegn/side-11/',
 'https://www.dba.dk/boliger/lejebolig/lejelejli

# Get all the links of all ads, from all pages

In [13]:
total_links_ads = []
for link in links_of_pages:
    url = link
    soup = BeautifulSoup(response.text,'html.parser')
    ads=soup.find_all("tr",{"class":["dbaListing listing topListing","dbaListing listing hasInsertionFee topListing","dbaListing listing hasInsertionFee","dbaListing listing lastListing","dbaListing listing"]})
    for i in range(0,len(ads)):
        total_links_ads.append(ads[i].find("a",{"class":"listingLink"})["href"])

In [14]:
len(total_links_ads)

2200

In [15]:
total_links_ads

['https://www.dba.dk/3500-3-vaer-lejlighed-69-m2/id-1110918131/',
 'https://www.dba.dk/1658-4-vaer-lejlighed-81-m2/id-1111184498/',
 'https://www.dba.dk/2300-1-vaer-lejlighed-18-m2/id-1111183325/',
 'https://www.dba.dk/3-vaerelses-lejlighed-i-koebe/id-509876410/',
 'https://www.dba.dk/4-vaer-eller-stoerre-i-koeben/id-509876407/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-koebe/id-510023270/',
 'https://www.dba.dk/4-vaer-eller-stoerre-i-konge/id-508806058/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-roedo/id-510023210/',
 'https://www.dba.dk/3-vaerelses-lejlighed-i-koebe/id-510023153/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-koebe/id-510023154/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-kong/id-510023141/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-kong/id-510023139/',
 'https://www.dba.dk/2-vaerelses-lejlighed-i-broen/id-510023136/',
 'https://www.dba.dk/1-vaerelses-lejlighed-i-koebe/id-510023119/',
 'https://www.dba.dk/3-vaerelses-lejlighed-i-hvid/id-510023112/',
 '

# Get ad data

In [16]:
# Need to iterate through total_links_ads list, connect to each link, store the data in a list that contains dictionaries

In [17]:
# Lets try for one ad

In [18]:
url = total_links_ads[0]
url

'https://www.dba.dk/3500-3-vaer-lejlighed-69-m2/id-1110918131/'

In [19]:
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [20]:
# Title of an ad
soup.find("div",{"class":"vip-heading"}).find_all('div',{"class":"row-fluid"})[0].get_text().replace('\n','').strip()

'3500 3 vær. lejlighed, 69 m2, Bymidten 56 56 2'

In [21]:
# Price
soup.find("div",{"class":"vip-heading"}).find_all('div',{"class":"row-fluid"})[1].get_text().replace('\n','').strip()

'11 kr.'

In [22]:
# Description
','.join(list(map(str,list(soup.find('div',{"class":"vip-additional-text"}).find_all('p'))))).translate(str.maketrans('','',"<p>\r\n</p>"))

'            3500 3 vær. lejlighed, 69 m2, Bymidten 56 56 2, 1 mdr forudbetalt leje, 33000 i deositum, overtagelse 14-2024, vaskemaskine, ovaskemaskine, arkeringslads, kælderrum        ,Nyrenoveret lejlighed brTæt å stationen og bymidten br3 værelser brPris 11.000 i leje'

In [23]:
# House features keys
h_keys = list(map(str,list(soup.find('div',{"class":"vip-matrix-data"}).find_all('dt'))))

In [24]:
# House features values
h_values = list(map(str,list(soup.find('div',{"class":"vip-matrix-data"}).find_all('dd'))))

In [25]:
# Zip key values 
feat_dict = {k:v for (k,v) in zip(h_keys,h_values)}
feat_dict

{'<dt>Boligtype</dt>': '<dd>lejlighed</dd>',
 '<dt>Adresse</dt>': '<dd>Bymidten 56 56 2</dd>',
 '<dt>Postnr.</dt>': '<dd>3500</dd>',
 '<dt>Antal værelser</dt>': '<dd>3</dd>',
 '<dt>Boligkvm.</dt>': '<dd>69</dd>',
 '<dt>Forudbetalt leje</dt>': '<dd>1</dd>',
 '<dt>Depositum</dt>': '<dd>33000</dd>',
 '<dt>Overtagelsesdato</dt>': '<dd>1/4-2024</dd>',
 '<dt>Fakta</dt>': '<dd>vaskemaskine, opvaskemaskine, parkeringsplads, kælderrum</dd>'}

# Get all ad data

In [26]:
# Need to iterate through total_links_ads list, connect to each link, store the data in a list that contains dictionaries

In [27]:
data = {}
def get_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    data['title'] = soup.find("div",{"class":"vip-heading"}).find_all('div',{"class":"row-fluid"})[0].get_text().replace('\n','').strip()
    data['price'] = soup.find("div",{"class":"vip-heading"}).find_all('div',{"class":"row-fluid"})[1].get_text().replace('\n','').strip()
    data['description'] = ','.join(list(map(str,list(soup.find('div',{"class":"vip-additional-text"}).find_all('p'))))).translate(str.maketrans('','',"<p>\r\n</p>"))
    h_keys = list(map(str,list(soup.find('div',{"class":"vip-matrix-data"}).find_all('dt'))))
    h_values = list(map(str,list(soup.find('div',{"class":"vip-matrix-data"}).find_all('dd'))))
    data['features'] = {k:v for (k,v) in zip(h_keys,h_values)}
    return data

In [28]:
all_data = []
for link in tqdm(total_links_ads):
    data = {}
    all_data.append(get_data(link))

  1%|█                                                                               | 28/2200 [00:38<49:38,  1.37s/it]


KeyboardInterrupt: 

In [ ]:
all_data

# Store data in dataframe

In [ ]:
df = pd.DataFrame(all_data)

In [ ]:
df